In [1]:
import sys
sys.path.append("..")
import torch
from torch.utils.data import DataLoader
from transformers import AutoFeatureExtractor, ASTForAudioClassification
from settings import ALL_CLASSES

In [2]:
dataset, labels = torch.load("C:/Users/Filip/Desktop/PW/2 semestr/Deep Learning/pro2/MINI_DL_RNN/src/artifacts/speech-waveform-v0/validation.pt")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Filip/Desktop/PW/2 semestr/Deep Learning/pro2/MINI_DL_RNN/src/artifacts/speech-waveform-v0/validation.pt'

In [ ]:
dataset.shape

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

In [3]:
from transformers import ASTConfig, ASTForAudioClassification

# Load the configuration of the AST model
config = ASTConfig.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

# Modify the configuration to adjust the number of labels
config.num_labels = 12  # Set this to the number of your classes

# Now load the model with the updated configuration
model = ASTForAudioClassification(config)


In [4]:
model

ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTAttention(
            (attention): ASTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
            (de

In [18]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import ASTConfig, ASTForAudioClassification, ASTFeatureExtractor
from tqdm import tqdm

# Assuming 'dataset' and 'labels' are already loaded
tensor_dataset = TensorDataset(dataset, labels)

# DataLoader
batch_size = 16
data_loader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=False)

# Feature Extractor
feature_extractor = ASTFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

# Load model and move to device
model.to(device)
model.eval()

# Function to preprocess, perform inference, and calculate accuracy
def validate_model(data_loader, model, device):
    all_predictions = []
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in tqdm(data_loader, desc="Validating", unit="batch"):
        # Convert waveform to log Mel spectrogram
        inputs = feature_extractor(inputs.numpy(), return_tensors="pt", sampling_rate=16000).input_values
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())

            # Calculate accuracy
            correct_predictions += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = correct_predictions / total_samples
    return all_predictions, accuracy

# Run validation
predictions, accuracy = validate_model(data_loader, model, device)
print("Validation Accuracy:", accuracy)


Validating: 100%|██████████| 419/419 [3:45:20<00:00, 32.27s/batch]  

Validation Accuracy: 0.5894988066825776


# Tu koniec

In [12]:
input_features = processor(
    dataset.numpy()[1005:1010], sampling_rate=16000, return_tensors="pt"
).input_features.to(device)

In [13]:
predicted_ids = model.generate(input_features)

In [14]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription

[' left', ' Left', ' Left.', ' Left.', ' Who That?']

In [15]:
[ALL_CLASSES[val] for val in labels[1005:1010].squeeze()]

['left', 'left', 'left', 'left', 'left']